# Histogram Gradient Boosting (GBMs) via Randomized Search CV # 

In [15]:
import numpy as np
import pandas
from sklearn import linear_model
from sklearn import metrics
import matplotlib.pyplot as plt

from sklearn.experimental import enable_hist_gradient_boosting  # explicitly require this experimental feature
from sklearn.ensemble import HistGradientBoostingClassifier

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score

import time

Run Histogram-based GBMs on TWO datasets: One for the 28x28 images, one for the 64x64 images.

## 28 x 28 images ##

In [16]:
data = pandas.read_csv('data28.csv')
y = data['label'].values
X = data[data.columns[1:]].values

# 70-30 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 1)

In [17]:
grid = {'max_iter' : 250 * np.arange(1, 13)} # 250, 500, ..., 3000
gbm=HistGradientBoostingClassifier(max_depth=4, learning_rate=0.01) # choose learning rate 0.01, depth 4
gbm_cv_=RandomizedSearchCV(gbm,grid,cv=5,n_jobs=-1,verbose=4,random_state=1,n_iter=5) # run all jobs in parallel, and print the CV accuracy per fold on the terminal (verbose = 4)
t0 = time.time()
gbm_cv_.fit(X_train,y_train)
t1 = time.time()
print('Randomized search (5 iterations) for 28 x 28 images took', t1 - t0, 'seconds')

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Randomized search (10 iterations) for 28 x 28 images took 1160.9488430023193 seconds


In [18]:
print("tuned hyperparameters: (best parameters) ",gbm_cv_.best_params_)
print("CV accuracy :",gbm_cv_.best_score_)

tuned hyperparameters: (best parameters)  {'max_iter': 2750}
CV accuracy : 0.9393333333333332


In [19]:
max_iter = gbm_cv_.best_params_['max_iter']

clf = HistGradientBoostingClassifier(max_depth=4, max_iter=max_iter, learning_rate=0.01)
t0 = time.time()
clf.fit(X_train, y_train)
t1 = time.time()
print('Training for 28 x 28 images took', t1 - t0, 'seconds')

Training for 28 x 28 images took 135.37225699424744 seconds


In [20]:
# training accuracy
train_acc = accuracy_score(y_train, clf.predict(X_train)) 
train_acc

1.0

In [21]:
# test accuracy
test_acc = accuracy_score(y_test, clf.predict(X_test)) 
test_acc

0.9611428571428572

In [22]:
# write cv results (such as mean fit times for each hyperparam configuration) to file
# make sure to run the above cells first before running this!!!!
with open('gbm_28_randomcv.txt', 'w') as file:
    file.write(str(gbm_cv_.cv_results_))

## 64 x 64 images ##

In [23]:
data = pandas.read_csv('data64.csv')
y = data['label'].values
X = data[data.columns[1:]].values

# 70-30 train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.7, random_state = 1)

In [26]:
grid = {'max_iter' : 250 * np.arange(1, 13)} # 250, 500, ..., 3000
gbm=HistGradientBoostingClassifier(max_depth=4, learning_rate=0.01) # choose learning rate 0.01, depth 4
gbm_cv=RandomizedSearchCV(gbm,grid,cv=5,n_jobs=-1,verbose=4,random_state=1,n_iter=5) # run all jobs in parallel, and print the CV accuracy per fold on the terminal (verbose = 4)
t0 = time.time()
gbm_cv.fit(X_train,y_train)
t1 = time.time()
print('Randomized search (5 iterations) for 64 x 64 images took', t1 - t0, 'seconds')

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Randomized search (5 iterations) for 64 x 64 images took 7646.927664041519 seconds


In [27]:
print("tuned hyperparameters: (best parameters) ",gbm_cv.best_params_)
print("CV accuracy :",gbm_cv.best_score_)

tuned hyperparameters: (best parameters)  {'max_iter': 2750}
CV accuracy : 0.9133333333333333


In [28]:
max_iter = gbm_cv.best_params_['max_iter']

clf = HistGradientBoostingClassifier(max_depth=4, max_iter=max_iter, learning_rate=0.01)
t0 = time.time()
clf.fit(X_train, y_train)
t1 = time.time()
print('Training for 64 x 64 images took', t1 - t0, 'seconds')

Training for 64 x 64 images took 708.2642109394073 seconds


In [29]:
# training accuracy
train_acc = accuracy_score(y_train, clf.predict(X_train)) 
train_acc

1.0

In [30]:
# test accuracy
test_acc = accuracy_score(y_test, clf.predict(X_test)) 
test_acc

0.9388571428571428

In [31]:
# write cv results (such as mean fit times for each hyperparam configuration) to file
with open('gbm_64_randomcv.txt', 'w') as file:
    file.write(str(gbm_cv.cv_results_))